In [113]:
import pyomo.environ as pyo

from scipy.spatial.distance import pdist, squareform

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import time
from itertools import cycle

#from pyomo.contrib.appsi.solvers import Highs
# import gurobipy
import pandas as pd

In [114]:
data = pd.read_csv('data.csv')

In [115]:
from_list = list(data['From'])
to_list = list(data['To'])
distance_list = list(data['Distance'])
cost_list = list(data['Cost'])

from_list.append(100)
to_list.append(1)
distance_list.append(0)
cost_list.append(0)
arcs = [(from_list[i], to_list[i]) for i in range(len(from_list))]
arcs


[(1, 37),
 (1, 59),
 (1, 72),
 (2, 15),
 (2, 20),
 (2, 26),
 (2, 37),
 (2, 58),
 (2, 60),
 (2, 76),
 (2, 80),
 (2, 90),
 (2, 95),
 (2, 100),
 (3, 15),
 (3, 37),
 (3, 40),
 (3, 41),
 (3, 56),
 (3, 59),
 (3, 73),
 (3, 76),
 (4, 7),
 (4, 29),
 (4, 46),
 (4, 61),
 (4, 80),
 (4, 83),
 (4, 98),
 (5, 11),
 (5, 34),
 (5, 55),
 (5, 79),
 (5, 80),
 (5, 85),
 (5, 87),
 (5, 92),
 (5, 99),
 (6, 8),
 (6, 15),
 (6, 24),
 (6, 30),
 (6, 31),
 (6, 35),
 (6, 39),
 (6, 40),
 (6, 41),
 (6, 51),
 (6, 74),
 (6, 82),
 (7, 6),
 (7, 8),
 (7, 10),
 (7, 24),
 (7, 25),
 (7, 35),
 (7, 47),
 (7, 52),
 (7, 74),
 (8, 6),
 (8, 16),
 (8, 18),
 (8, 21),
 (8, 25),
 (8, 26),
 (8, 39),
 (8, 46),
 (8, 55),
 (8, 58),
 (8, 63),
 (8, 83),
 (9, 5),
 (9, 18),
 (9, 22),
 (9, 48),
 (9, 60),
 (9, 79),
 (9, 92),
 (10, 13),
 (10, 15),
 (10, 31),
 (10, 37),
 (10, 85),
 (10, 88),
 (10, 93),
 (10, 96),
 (11, 1),
 (11, 2),
 (11, 13),
 (11, 15),
 (11, 22),
 (11, 24),
 (11, 34),
 (11, 48),
 (11, 53),
 (11, 54),
 (11, 55),
 (11, 61),
 (11, 6

In [116]:
cost_dict ={}
for idx, arc in enumerate(arcs):
    cost_dict[arc] = cost_list[idx]
cost_dict

{(1, 37): 5,
 (1, 59): 59,
 (1, 72): 1,
 (2, 15): 1,
 (2, 20): 39,
 (2, 26): 6,
 (2, 37): 36,
 (2, 58): 8,
 (2, 60): 9,
 (2, 76): 9,
 (2, 80): 21,
 (2, 90): 1,
 (2, 95): 26,
 (2, 100): 10,
 (3, 15): 0,
 (3, 37): 6,
 (3, 40): 1,
 (3, 41): 9,
 (3, 56): 24,
 (3, 59): 20,
 (3, 73): 11,
 (3, 76): 4,
 (4, 7): 9,
 (4, 29): 24,
 (4, 46): 17,
 (4, 61): 6,
 (4, 80): 3,
 (4, 83): 15,
 (4, 98): 5,
 (5, 11): 19,
 (5, 34): 1,
 (5, 55): 5,
 (5, 79): 7,
 (5, 80): 11,
 (5, 85): 77,
 (5, 87): 14,
 (5, 92): 13,
 (5, 99): 14,
 (6, 8): 1,
 (6, 15): 16,
 (6, 24): 45,
 (6, 30): 10,
 (6, 31): 20,
 (6, 35): 17,
 (6, 39): 7,
 (6, 40): 1,
 (6, 41): 17,
 (6, 51): 12,
 (6, 74): 14,
 (6, 82): 10,
 (7, 6): 15,
 (7, 8): 37,
 (7, 10): 1,
 (7, 24): 5,
 (7, 25): 2,
 (7, 35): 53,
 (7, 47): 11,
 (7, 52): 103,
 (7, 74): 10,
 (8, 6): 29,
 (8, 16): 4,
 (8, 18): 12,
 (8, 21): 14,
 (8, 25): 13,
 (8, 26): 2,
 (8, 39): 9,
 (8, 46): 19,
 (8, 55): 12,
 (8, 58): 8,
 (8, 63): 39,
 (8, 83): 13,
 (9, 5): 15,
 (9, 18): 3,
 (9, 22): 1,


In [117]:
distance_dict ={}
for idx, arc in enumerate(arcs):
    distance_dict[arc] = distance_list[idx]
distance_dict

{(1, 37): 60,
 (1, 59): 9,
 (1, 72): 63,
 (2, 15): 61,
 (2, 20): 16,
 (2, 26): 54,
 (2, 37): 26,
 (2, 58): 54,
 (2, 60): 25,
 (2, 76): 72,
 (2, 80): 35,
 (2, 90): 73,
 (2, 95): 23,
 (2, 100): 20,
 (3, 15): 30,
 (3, 37): 51,
 (3, 40): 75,
 (3, 41): 19,
 (3, 56): 27,
 (3, 59): 17,
 (3, 73): 84,
 (3, 76): 78,
 (4, 7): 58,
 (4, 29): 42,
 (4, 46): 38,
 (4, 61): 33,
 (4, 80): 61,
 (4, 83): 49,
 (4, 98): 20,
 (5, 11): 37,
 (5, 34): 28,
 (5, 55): 84,
 (5, 79): 76,
 (5, 80): 62,
 (5, 85): 10,
 (5, 87): 43,
 (5, 92): 57,
 (5, 99): 28,
 (6, 8): 22,
 (6, 15): 63,
 (6, 24): 16,
 (6, 30): 68,
 (6, 31): 13,
 (6, 35): 24,
 (6, 39): 58,
 (6, 40): 77,
 (6, 41): 17,
 (6, 51): 75,
 (6, 74): 52,
 (6, 82): 87,
 (7, 6): 37,
 (7, 8): 23,
 (7, 10): 97,
 (7, 24): 53,
 (7, 25): 59,
 (7, 35): 13,
 (7, 47): 75,
 (7, 52): 6,
 (7, 74): 54,
 (8, 6): 22,
 (8, 16): 92,
 (8, 18): 59,
 (8, 21): 62,
 (8, 25): 74,
 (8, 26): 87,
 (8, 39): 78,
 (8, 46): 49,
 (8, 55): 40,
 (8, 58): 42,
 (8, 63): 21,
 (8, 83): 49,
 (9, 5): 28,

In [118]:
model = pyo.ConcreteModel()

## Sets

In [119]:
model.A = pyo.Set(initialize=[(from_list[i], to_list[i]) for i in range(len(from_list))]) #set of arcs
model.N = pyo.Set(initialize=range(1,101)) # set of all places 
model.D = pyo.Set(initialize=range(2,100))  #set of places to 

In [120]:
model.D.pprint()

D : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   98 : {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}


## Parameters

## Variables

In [121]:
model.X = pyo.Var(model.A, within=pyo.Binary) # decision to move along arc ij 
model.Y = pyo.Var(model.N, within=pyo.Binary) # decision of if node N is visited

In [122]:
def visit_once_rule(model,i): #2/13
    return model.Y[i] <= 1
model.visit_once_rule = pyo.Constraint(model.N, rule=visit_once_rule)

def arcs_out_rule(model, i): #3/14 modifyied Arcs out rule
    return sum(model.X[i, j] for j in model.N if (i,j) in arcs) == model.Y[i]
model.arcs_out_rule = pyo.Constraint(model.N, rule=arcs_out_rule)

def arcs_in_rule(model, i): #3/14 modifyied Arcs in rule
    return sum(model.X[j, i] for j in model.N if (j,i) in arcs) == model.Y[i]
model.arcs_in_rule = pyo.Constraint(model.N, rule=arcs_in_rule)

def visit_start(model): 
    return model.Y[1] == 1
model.visit_start = pyo.Constraint(rule=visit_start)

def visit_end(model): 
    return model.Y[100] == 1
model.visit_end = pyo.Constraint(rule=visit_end)

def cost(model):
    return sum(model.X[i,j] * cost_dict[i,j] for (i, j) in model.A) <= 73
model.cost = pyo.Constraint(rule=cost)

## Obj

In [123]:
model.obj = pyo.Objective(
    expr=sum(
        model.X[i,j] * distance_dict[i,j]
        for (i, j) in model.A
        ), 
    sense=pyo.minimize,
)

In [124]:
TIME_LIMIT = 60
THREADS = 4

SOVLER_ENGINE = 'appsi_highs'
#solvers glpk  appsi_highs cplex gurobi gurobi_persistent


solver = pyo.SolverFactory(SOVLER_ENGINE)

if SOVLER_ENGINE == 'cbc':
        solver.options['seconds'] = TIME_LIMIT
elif SOVLER_ENGINE == 'glpk':
        solver.options['tmlim'] = TIME_LIMIT
elif SOVLER_ENGINE == 'appsi_highs':
        solver.options['time_limit'] = TIME_LIMIT
        #solver.options['parallel'] = True
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'cplex':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi_persistent':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
        solver.set_instance(model) # remove model from  solver.solve(tee=True)
        solver.set_gurobi_param("PoolSolutions", 500)
        solver.set_gurobi_param("PoolSearchMode", 0)
# sol = solver.solve(tee= True)
sol = solver.solve(model, tee= True) #, warmstart=True , logfile= 'log.txt'

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+02]
  Cost   [1e+00, 1e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 7e+01]
Presolving model
201 rows, 1037 cols, 3011 nonzeros  0s
200 rows, 1032 cols, 2991 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   200 rows
   1032 cols (1032 binary, 0 integer, 0 implied int., 0 continuous)
   2991 nonzeros
MIP-Timing:       0.062 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol      

In [125]:
print(sol.solver.status)
print(sol.solver.termination_condition)
print (model.obj())
print ("Cost:", sum(model.X[i,j].value * cost_dict[i,j] for (i, j) in model.A))

ok
optimal
131.0
Cost: 44.0


In [126]:
model.display()

Model unknown

  Variables:
    X : Size=956, Index=A
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
          (1, 37) :     0 :   1.0 :     1 : False : False : Binary
          (1, 59) :     0 :   0.0 :     1 : False : False : Binary
          (1, 72) :     0 :   0.0 :     1 : False : False : Binary
          (2, 15) :     0 :   0.0 :     1 : False : False : Binary
          (2, 20) :     0 :   0.0 :     1 : False : False : Binary
          (2, 26) :     0 :   0.0 :     1 : False : False : Binary
          (2, 37) :     0 :   0.0 :     1 : False : False : Binary
          (2, 58) :     0 :   0.0 :     1 : False : False : Binary
          (2, 60) :     0 :   0.0 :     1 : False : False : Binary
          (2, 76) :     0 :   0.0 :     1 : False : False : Binary
          (2, 80) :     0 :   0.0 :     1 : False : False : Binary
          (2, 90) :     0 :   0.0 :     1 : False : False : Binary
          (2, 95) :     0 :   0.0 :     1 : False : False : Binary
        

In [134]:
node = 1
for i in model.N:
    for j in model.N:
        if (node,j) in arcs:
            if np.isclose(model.X[node, j].value, 1, atol=1e-1):
                print (node, " --", j)
                node = j
                break
    if node == 1:
        break


1  -- 37
37  -- 41
41  -- 2
2  -- 100
100  -- 1
